In [1]:
#importar bibliotecas
import pandas as pd
import pathlib
import win32com.client as win32

#importar base de dados 
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding= 'latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

#incluir nome da loja em vendas
vendas = vendas.merge(lojas, on='ID Loja')

dicionario_lojas = {}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]

data_indicador = vendas['Data'].max()


#identificar se a pasta ja existe

caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja 
        nova_pasta.mkdir()
        
        
    #salvar dentro da pasta
    nome_arquivo = '{}_{}_{}.xlsx'.format(data_indicador.month, data_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)    
    
    
#definição de metas

meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500


for loja in dicionario_lojas:
    
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==data_indicador, :]

    #faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    #print(faturamento_ano)
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    #print(faturamento_dia)

    #diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    #print(qtde_produtos_ano)
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    #print(qtde_produtos_dia)
    
    
    #ticket médio
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    #print(ticket_medio_ano)
    #ticket_medio_dia
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    #print(ticket_medio_dia)
    
    
    #enviar o e-mail
    outlook = win32.Dispatch('outlook.application')

    dia = data_indicador.day
    mes = data_indicador.month

    nome =  emails.loc[emails['Loja']==loja, 'Gerente'].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia}/{mes} - loja {loja}'
    #mail.Body = 'Texto do E-mail'

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if faturamento_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano          
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_tk_dia =  'green'
    else:       
        cor_tk_dia = 'red'
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_tk_ano =  'green'
    else:       
        cor_tk_ano = 'red'

    mail.HTMLBody = f'''
    <p>Bom dia, {nome}</p>
    <p>O resultado de ontem <strong>({dia}/{mes})</strong> da <strong>loja {loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Idicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color={cor_fat_dia}>◙</td>
      </tr>
      <tr>
         <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font color={cor_qtde_dia}>◙</td>
      </tr>
      <tr>
         <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color={cor_tk_dia}>◙</td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Idicador</th>
        <th>Valor ano</th>
        <th>Meta ano</th>
        <th>Cenário ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color={cor_fat_ano}>◙</td>
      </tr>
      <tr>
         <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center"><font color={cor_qtde_ano}>◙</td>
      </tr>
      <tr>
         <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font color={cor_tk_ano}>◙</td>
      </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>
    <p>Qualquer duvida estou a disposição.</p>
    <p>att., Weberson</p>
    '''
    # Anexos (pode colocar quantos quiser):
    attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{data_indicador.month}_{data_indicador.day}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    mail.Send()
    print('E-mail da Loja {} enviado'.format(loja))
    
    
#Criar ranking para diretoria

faturamento_lojas = vendas.groupby('Loja')[['Loja','Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(mes, dia)
faturamento_lojas.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

vendas_dia = vendas.loc[vendas['Data']==data_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja','Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(mes, dia)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))    


#enviar e-mail para diretoria

outlook = win32.Dispatch('outlook.application')

dia = data_indicador.day
mes = data_indicador.month

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia}/{mes}'
mail.Body = f'''
Prezados, Bom dia

Melhor loja do dia em faturamento: Loja {faturamento_lojas_dia.index[0]} com faturamento de: R${faturamento_lojas_dia.iloc[0, 0]:.2f}
Pior loja do dia em faturamento: Loja {faturamento_lojas_dia.index[-1]} com faturamento de: R${faturamento_lojas_dia.iloc[-1, 0]:.2f}

Melhor loja do ano em faturamento: Loja {faturamento_lojas_ano.index[0]} com faturamento de: R${faturamento_lojas_ano.iloc[0, 0]:.2f}
Pior loja do ano em faturamento: Loja {faturamento_lojas_ano.index[-1]} com faturamento de: R${faturamento_lojas_ano.iloc[-1, 0]:.2f}

Segue em anexo os rankings do ano e do dia de todas as lojas.

Qualquer duvida estou á disposição

att.,
Weberson
'''

# Anexos (pode colocar quantos quiser):
attachment = pathlib.Path.cwd() / caminho_backup / f'{data_indicador.month}_{data_indicador.day}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))
attachment = pathlib.Path.cwd() / caminho_backup / f'{data_indicador.month}_{data_indicador.day}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))

mail.Send()
print('E-mail da Diretoria enviado')


E-mail da Loja Iguatemi Esplanada enviado
E-mail da Loja Shopping Midway Mall enviado
E-mail da Loja Norte Shopping enviado
E-mail da Loja Shopping Iguatemi Fortaleza enviado
E-mail da Loja Shopping União de Osasco enviado
E-mail da Loja Shopping Center Interlagos enviado
E-mail da Loja Rio Mar Recife enviado
E-mail da Loja Salvador Shopping enviado
E-mail da Loja Rio Mar Shopping Fortaleza enviado
E-mail da Loja Shopping Center Leste Aricanduva enviado
E-mail da Loja Ribeirão Shopping enviado
E-mail da Loja Shopping Morumbi enviado
E-mail da Loja Parque Dom Pedro Shopping enviado
E-mail da Loja Bourbon Shopping SP enviado
E-mail da Loja Palladium Shopping Curitiba enviado
E-mail da Loja Passei das Águas Shopping enviado
E-mail da Loja Center Shopping Uberlândia enviado
E-mail da Loja Shopping Recife enviado
E-mail da Loja Shopping Vila Velha enviado
E-mail da Loja Shopping SP Market enviado
E-mail da Loja Shopping Eldorado enviado
E-mail da Loja Shopping Ibirapuera enviado
E-mail da L